In [1]:
import collections
import copy
import datetime
import itertools
import sys

import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
show_Ids = True
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay
Tournament = cibblbibbl.tournament.handler.default.Tournament
NormalPlayer = cibblbibbl.player.NormalPlayer
StarPlayer = cibblbibbl.player.StarPlayer

In [3]:
G.init()

In [4]:
swithout = max(G.seasons).standings_without_partnership()
swithout

[[995, (160, 190, 210, 80, 60, 50), Team(867862)],
 [975, (200, 170, 130, 35, 110, 50), Team(876774)],
 [930, (40, 150, 280, 100, 40, 80), Team(886114)],
 [920, (140, 140, 260, 110, 25, 35), Team(854678)],
 [860, (260, 170, 80, 120, 70, 40), Team(897376)],
 [845, (50, 160, 230, 105, 55, 60), Team(895929)],
 [840, (110, 170, 210, 30, 80, 50), Team(878945)],
 [805, (150, 190, 180, 75, 50, 100), Team(874545)],
 [800, (110, 170, 170, 110, 45, 50), Team(854257)],
 [785, (50, 220, 140, 40, 80, 0), Team(871787)],
 [770, (80, 170, 190, 115, 5, 50), Team(903654)],
 [765, (50, 170, 230, 75, 70, 35), Team(892248)],
 [755, (60, 140, 50, 50, 100, 130), Team(874452)],
 [720, (110, 110, 120, 15, 70, 40), Team(888994)],
 [710, (60, 120, 50, 55, 70, 125), Team(854600)],
 [690, (180, 80, 80, 30, 100, 110), Team(886002)],
 [690, (120, 40, 200, 110, 35, 0), Team(910344)],
 [690, (60, 120, 160, 35, 125, 20), Team(879131)],
 [680, (190, 140, 60, 110, 20, 60), Team(874966)],
 [680, (110, 90, 200, 35, 35, 15)

In [ ]:
swith = max(G.seasons).standings_with_partnership()
swith

In [ ]:
gold_teams = 

In [6]:
dswithout = {Te: [nr, prestige] for nr, (prestige, ties, Te) in enumerate(swithout, 1)}

In [7]:
LA = []
for nr1, (prestige1, ties, Te) in enumerate(swith, 1):
    nr0, prestige0 = dswithout[Te]
    LL = [str(Te), nr1, nr0, prestige1, prestige0]
    if prestige0 + 50 == prestige1:
        LL[0] = "(G) " + LL[0]
    elif prestige0 + 25 == prestige1:
        LL[0] = "(S) " + LL[0]
    LA.append(LL)

In [8]:
table = texttable.Texttable()
table.set_cols_dtype(["t", "i", "i", "i", "i"])
table.set_cols_align(["l", "r", "r", "r", "r"])
table.set_cols_width([34, 3, 3, 4, 4])
table.set_deco(
    texttable.Texttable.HEADER
    #| texttable.Texttable.VLINES
)
table.add_rows([["Name", "PoP", "PoN", "PrP", "PrN"],] + LA)
print(table.draw())

               Name                  PoP   PoN   PrP    PrN 
(G) Everlasting Vultures of Settra     1     1   1045    995
(S) Congregation of Holy Touchdown     2     2   1000    975
(G) Lords of the Silent Death          3     4    970    920
(S) Eye and Arrow                      4     3    955    930
(S) Darkenwood Sentinels               5     5    885    860
(S) Fangsnapper's Clawpack             6     6    870    845
(S) Eternal Ecstasy                    7     7    865    840
(G) Grand Celestial Delegate           8     9    850    800
(S) Stone Prince's Men                 9     8    830    805
(S) Scions of Neferata                10    11    795    770
(S) Crimson Fins                      11    12    790    765
Malignant Curators                    12    10    785    785
(S) Way of the Dragon                 13    13    780    755
(G) Montetriste Noblemen              14    15    760    710
(S) Beerbog Behemoths                 15    14    745    720
(G) Bhubaneswar Rampage 

In [ ]:
def tooltips(self):
  if self._tooltips is ...:
    self._tooltips = []
    for sh in self.rostershorthands:
      rosterId = self._rostershorthands[sh]
      roster = sr.roster.Roster(rosterId)
      self._tooltips.append(
          f'[block=tooltip id={sh}]{roster.name}[/block]'
      )
  return self._tooltips

In [69]:
gold_url = "/FUMBBL/Images/gold_small.png"
silver_url = "/FUMBBL/Images/silver_small.png"

In [70]:
gold_partner_teams = max(G.seasons).gold_partner_teams()
silver_partner_teams = max(G.seasons).silver_partner_teams()

In [77]:
rows = []
tooltip_n = 1
tooltips = []
d_tooltip_n = {}
prev_prestige = None
prev_ties = None
for nr, (prestige, ties, Te) in enumerate(swith, 1):
    if ties in d_tooltip_n:
        n = d_tooltip_n[ties]
    else:
        tooltip_n += 1
        n = tooltip_n
        d_tooltip_n[ties] = n
    if (prestige, ties) == (prev_prestige, prev_ties):
        nrstr = ""
    else:
        nrstr = f'{nr}'
    BBprestige = f'[block tooltip={n}]{prestige}[/block]'
    tooltips.append(f'[block=tooltip id={n}]Team prestiges (tiebreaker): {", ".join(str(x) for x in ties)}[/block]\\')
    BBTe = cibblbibbl.bbcode.url(f'/p/team?id={Te.Id}', Te.name)
    if Te in gold_partner_teams:
        BBTe = cibblbibbl.bbcode.img(gold_url) + " " + BBTe
    elif Te in silver_partner_teams:
        BBTe = cibblbibbl.bbcode.img(silver_url) + " " + BBTe
    BBCoach = cibblbibbl.bbcode.url(f'/~{Te.coach_name}', Te.coach_name)
    rows.append([nrstr, BBprestige, BBTe, Te.roster_name, BBCoach])
    prev_prestige = prestige
    prev_ties = ties

In [78]:
align = ["R", "R", "L", "L", "L"]
header = ["#", "Pres.", "Team", "Roster", "Coach"]
width = "600px"

In [79]:
bbcode_table = cibblbibbl.bbcode.table(rows, align=align, header=header, width=width)

In [80]:
s = f'''[block][/block]

[block=automargin width={width}]\\
{bbcode_table}\\
[/block]
{cibblbibbl.bbcode.N.join(tooltips)}

[block][/block]
'''

In [81]:
print(s)

[block][/block]

[block=automargin width=600px]\
[table blackborder border2 width=600px]\
	[tr bg=black fg=white]\
		[td][block=center]#[/block][/td]\
		[td][block=center]Pres.[/block][/td]\
		[td][block=center]Team[/block][/td]\
		[td][block=center]Roster[/block][/td]\
		[td][block=center]Coach[/block][/td]\
	[/tr]\
	[tr bg=#e6ddc7]\
		[td][block=right]1[/block][/td]\
		[td][block=right][block tooltip=2]1045[/block][/block][/td]\
		[td][img]/FUMBBL/Images/gold_small.png[/img] [url=/p/team?id=867862]Everlasting Vultures of Settra[/url][/td]\
		[td]Zandri[/td]\
		[td][url=/~DatMonsta]DatMonsta[/url][/td]\
	[/tr]\
	[tr bg=#d6cdb7]\
		[td][block=right]2[/block][/td]\
		[td][block=right][block tooltip=3]1000[/block][/block][/td]\
		[td][img]/FUMBBL/Images/silver_small.png[/img] [url=/p/team?id=876774]Congregation of Holy Touchdown[/url][/td]\
		[td]Sisters of Sigmar[/td]\
		[td][url=/~Fanky]Fanky[/url][/td]\
	[/tr]\
	[tr bg=#e6ddc7]\
		[td][block=right]3[/block][/td]\
		[td][block=right][b